# Interactive Financial Analysis Report

This notebook provides an interactive way to perform a comprehensive financial analysis on a specified company. By leveraging a suite of intelligent agents, it can generate:
- **Fundamental Financial Analysis:** Including key ratios, Discounted Cash Flow (DCF) valuation, Enterprise Value, and an AI-generated assessment of financial health.
- **Shared National Credit (SNC) Analysis:** A regulatory assessment providing an SNC rating and rationale.
- **Strategic Insights:** Potential strategic opportunities and solutions identified by the Catalyst agent.
- **Consolidated Report:** A human-readable summary compiling all findings.

This tool is designed for financial analysts, investment professionals, and anyone interested in gaining automated insights into company performance and strategic positioning. 

Please provide your inputs in the cells below and click "Run Analysis" to begin.

## 1. Setup & Configuration

In [ ]:
# Code cell for imports and any necessary path setup (for finding cacm_adk_core)
import os
import sys
import json
import asyncio
import pandas as pd # Added pandas
import ipywidgets as widgets
from IPython.display import display, Markdown, JSON, clear_output

# Add project root to sys.path to allow imports from cacm_adk_core
# This assumes the notebook is in 'notebooks/' and the project root is one level up.
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import ADK components (will be used in later cells)
from cacm_adk_core.orchestrator.orchestrator import Orchestrator
from cacm_adk_core.semantic_kernel_adapter import KernelService
# from cacm_adk_core.context.shared_context import SharedContext # Orchestrator creates this

print("Setup cell executed. Libraries imported and path configured (if needed).")
print(f"Project root (expected): {project_root}")
# Add a check to see if a key module is importable
try:
    from cacm_adk_core.agents.base_agent import Agent
    print("Successfully imported a test class from cacm_adk_core. Path setup seems OK.")
except ImportError as e:
    print(f"ERROR: Could not import from cacm_adk_core. Path setup might be incorrect: {e}")
    print(f"Current sys.path: {sys.path}")

## 2. User Inputs

Provide the following inputs to customize your analysis:

*   **`Company ID`**: The stock ticker or a unique identifier for the company you wish to analyze.
    *   Full detailed data is available for `MSFT` and `TESTCORP`.
    *   For any other ID, the analysis will run using a *generic placeholder financial dataset*. The results will demonstrate the analytical process but will **not** be financially accurate for that specific company.
*   **`FAA Summary Guidance` (Optional)**: Provide specific instructions or questions to guide the textual summary generated by the FundamentalAnalystAgent.
    *   If left blank, a comprehensive default prompt is used to generate the summary.
    *   Example: "Focus on the company's debt structure and its impact on profitability." or "What are the key growth drivers and associated risks for the cloud segment?"
*   **`DCF Discount Rate Override` (Optional)**: Enter a decimal value (e.g., `0.09` for 9%) to override the default discount rate used in the Discounted Cash Flow (DCF) valuation by the FundamentalAnalystAgent.
    *   If set to `0.0` or left effectively zero, the agent will use its default assumptions or data-derived rates.
*   **`DCF Terminal Growth Override` (Optional)**: Enter a decimal value (e.g., `0.025` for 2.5%) to override the default terminal growth rate for the DCF valuation.
    *   If set to `0.0` or left effectively zero, the agent uses its defaults.

In [ ]:
# Define widgets
company_id_widget = widgets.Text(
    value='MSFT',
    placeholder='Enter Company ID (e.g., MSFT)',
    description='Company ID:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

faa_summary_guidance_widget = widgets.Textarea(
    value='',
    placeholder='Optional: e.g., Focus on cloud segment performance and AI strategy impact.',
    description='FAA Summary Guidance:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='80px')
)

dcf_discount_rate_override_widget = widgets.FloatText(
    value=0.0, # Using 0.0 as a sentinel for no override, agent logic will need to check for > 0
    description='DCF Discount Rate Override (e.g., 0.085):',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

dcf_terminal_growth_rate_override_widget = widgets.FloatText(
    value=0.0, # Using 0.0 as a sentinel for no override
    description='DCF Terminal Growth Override (e.g., 0.025):',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

# Display widgets
display(company_id_widget)
display(faa_summary_guidance_widget)
display(dcf_discount_rate_override_widget)
display(dcf_terminal_growth_rate_override_widget)

print("User input widgets displayed. Please enter your parameters above.")

## 3. Execution Engine

Clicking the "Run Analysis" button below will:
1.  Gather all the inputs you've specified above.
2.  Dynamically construct a workflow for our suite of analytical agents.
3.  Invoke the Orchestrator to execute this workflow.
4.  Capture the results, which will be displayed in the "Results Display" section.

Please be patient, as the full analysis may take a moment to complete. Logs will appear below the button during execution.

In [ ]:
# Global variable to store results for later display cells
latest_analysis_results = None

run_button = widgets.Button(description="Run Analysis")
output_area = widgets.Output() # To display logs during run and final raw JSON output

def on_run_button_clicked(b):
    global latest_analysis_results # To store results for other cells
    with output_area:
        clear_output(wait=True)
        print("Analysis started... Please wait.")

        # 1. Retrieve input values from widgets
        company_id_val = company_id_widget.value
        faa_summary_guidance_val = faa_summary_guidance_widget.value
        dcf_discount_override_val = dcf_discount_rate_override_widget.value
        dcf_terminal_growth_override_val = dcf_terminal_growth_rate_override_widget.value

        print(f"Company ID: {company_id_val}")
        print(f"FAA Summary Guidance: '{faa_summary_guidance_val}'")
        print(f"DCF Discount Rate Override: {dcf_discount_override_val if dcf_discount_override_val > 0 else 'Not overridden'}")
        print(f"DCF Terminal Growth Override: {dcf_terminal_growth_override_val if dcf_terminal_growth_override_val > 0 else 'Not overridden'}")
        print("\nConstructing CACM workflow...")

        # 2. Dynamically construct cacm_instance_data
        #    Based on examples/msft_comprehensive_analysis_workflow.json
        cacm_template = {
          "cacmId": f"interactive_analysis_{company_id_val.lower()}_{pd.Timestamp.now().strftime('%Y%m%d%H%M%S')}",
          "name": f"Interactive Analysis for {company_id_val}",
          "description": "Dynamically generated CACM for interactive analysis.",
          "inputs": {
            "target_company_id": {"value": company_id_val, "description": "Target company ID"},
            "msft_business_overview_text": { # Will be MSFT's for now, or generic if not MSFT
              "value": "Placeholder Business Overview. DataIngestionAgent should ideally fetch/select this based on company_id if not MSFT.",
              "description": "Business overview text."
            },
            "msft_risk_factors_text": { # Will be MSFT's for now
              "value": "Placeholder Risk Factors. DataIngestionAgent should ideally fetch/select this based on company_id if not MSFT.",
              "description": "Risk factors text."
            },
            "catalyst_input_params": {
              "value": {
                "client_id": f"{company_id_val}_Strategy_Team",
                "company_id": company_id_val,
                "industry": "DynamicIndustryContext", # Placeholder, could be another input
                "task_description_catalyst": f"Evaluate strategic opportunities for {company_id_val}."
              }
            },
            "report_generation_title_detail": {
                "value": f"Interactive Analysis for {company_id_val}",
            },
            # --- Inputs for FAA specific enhancements ---
            "user_faa_summary_guidance": {"value": faa_summary_guidance_val, "description": "User guidance for FAA summary."},
            "user_dcf_discount_rate": {"value": dcf_discount_override_val if dcf_discount_override_val > 0 else None, "description": "User DCF discount rate override."},
            "user_dcf_terminal_growth_rate": {"value": dcf_terminal_growth_override_val if dcf_terminal_growth_override_val > 0 else None, "description": "User DCF terminal growth override."}
          },
          "outputs": {
            "final_generated_report": {"type": "string"},
            "raw_fundamental_analysis": {"type": "object"},
            "raw_snc_analysis": {"type": "object"},
            "raw_catalyst_output": {"type": "object"}
          },
          "workflow": [
            {
              "stepId": "step1_ingest_data",
              "description": "Ingest specific text data and Catalyst parameters into SharedContext.",
              "computeCapabilityRef": "urn:adk:capability:standard_data_ingestor:v1",
              "inputBindings": {
                "companyName": {"value": f"{company_id_val} Corp."} , # Derived
                "companyTicker": {"value": company_id_val},
                "riskFactorsText": "cacm.inputs.msft_risk_factors_text", # Still using MSFT text for now
                "mockStructuredFinancialsForLLMSummary": "cacm.inputs.msft_business_overview_text" # Still using MSFT text
              }
            },
            {
              "stepId": "step2_fundamental_analysis",
              "description": "Perform fundamental financial analysis.",
              "computeCapabilityRef": "urn:adk:capability:fundamental_analyst_agent:v1",
              "inputBindings": {
                "company_id": "cacm.inputs.target_company_id",
                # Bindings for new FAA inputs (need to be added to FAA's capability definition in Step 6)
                "summary_guidance_prompt_addon": "cacm.inputs.user_faa_summary_guidance",
                "dcf_override_discount_rate": "cacm.inputs.user_dcf_discount_rate",
                "dcf_override_terminal_growth_rate": "cacm.inputs.user_dcf_terminal_growth_rate"
              },
              "outputBindings": {"analysis_result": "cacm.outputs.raw_fundamental_analysis"}
            },
            {
              "stepId": "step3_snc_analysis",
              "description": "Perform SNC analysis.",
              "computeCapabilityRef": "urn:adk:capability:snc_analyst_agent:v1",
              "inputBindings": {"company_id": "cacm.inputs.target_company_id"},
              "outputBindings": {"snc_analysis_result": "cacm.outputs.raw_snc_analysis"}
            },
            {
              "stepId": "step4_catalyst_insights",
              "description": "Generate strategic insights using CatalystWrapperAgent.",
              "computeCapabilityRef": "urn:adk:capability:catalyst_wrapper_agent:v1",
              "inputBindings": {
                "client_id": "cacm.inputs.catalyst_input_params.value.client_id",
                "company_id": "cacm.inputs.catalyst_input_params.value.company_id",
                "industry": "cacm.inputs.catalyst_input_params.value.industry"
              },
              "outputBindings": {"catalyst_output": "cacm.outputs.raw_catalyst_output"}
            },
            {
              "stepId": "step5_generate_report",
              "description": "Compile all analysis into a final report.",
              "computeCapabilityRef": "urn:adk:capability:standard_report_generator:v1",
              "inputBindings": {
                "report_title_detail": "cacm.inputs.report_generation_title_detail",
                "fundamental_analysis_data_ref": "cacm.outputs.raw_fundamental_analysis",
                "snc_analysis_data_ref": "cacm.outputs.raw_snc_analysis",
                "catalyst_data_ref": "cacm.outputs.raw_catalyst_output"
              },
              "outputBindings": {"final_report_text": "cacm.outputs.final_generated_report"}
            }
          ]
        }
        
        # For MSFT and TESTCORP, the DataRetrievalAgent has specific data.
        # For others, it will return "Data not found", which will propagate.
        # The Business Overview & Risk Factors are still hardcoded to MSFT's for this demo.
        # A more advanced version would have DataIngestionAgent fetch these based on company_id.
        if company_id_val not in ["MSFT", "TESTCORP"]:
            warning_message = (
                f"WARNING: Specific financial data for '{company_id_val}' is not pre-loaded. "
                f"The analysis will proceed using a GENERIC placeholder dataset. "
                f"Results will be illustrative of the analytical process but not reflective of '{company_id_val}' actual financials."
            )
            print(warning_message)
            # Update text inputs to be generic if not MSFT/TESTCORP
            cacm_template["inputs"]["msft_business_overview_text"]["value"] = f"Generic Business Overview for {company_id_val}. (Specific details would typically be company-dependent)."
            cacm_template["inputs"]["msft_risk_factors_text"]["value"] = f"Generic Risk Factors for {company_id_val}. (Risks are typically related to market dynamics, competition, operational factors, and financial leverage)."


        print("CACM instance constructed. Initializing Orchestrator...")
        try:
            kernel_service = KernelService() # Assuming default config
            orch = Orchestrator(kernel_service=kernel_service)
            # Orchestrator's __main__ has a mock validator. If that's not run, ensure validator is handled.
            if orch.validator is None: # Basic check, might need more robust validator setup
                from cacm_adk_core.validator.validator import Validator
                # Assuming a schema path or that default validator is okay for this dynamic instance
                # For now, let's ensure it doesn't crash if validator is strictly needed.
                # This might require providing a schema that matches this dynamic structure, or a permissive validator.
                # For simplicity, let's use a mock permissive validator if none is loaded.
                class MockPermissiveValidator:
                    schema = True 
                    def validate_cacm_against_schema(self, data): return True, []
                orch.validator = MockPermissiveValidator()
                print("Orchestrator validator was None, using MockPermissiveValidator for this run.")


            print("Orchestrator initialized. Running CACM workflow...")
            
            async def do_run_cacm():
                return await orch.run_cacm(cacm_template)

            if sys.platform == "win32" and sys.version_info >= (3, 8, 0): # Specific fix for ProactorLoop on Windows
                 asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
            
            success, logs, outputs = asyncio.run(do_run_cacm())

            latest_analysis_results = outputs # Store for display in other cells
            
            print("\n--- Orchestrator Execution Logs ---")
            for log_entry in logs:
                print(log_entry)
            print("\n--- Orchestrator Final Outputs (JSON) ---")
            # Pretty print JSON to the output area for easy reading
            print(json.dumps(outputs, indent=2))

            if success:
                print("\nSUCCESS: CACM workflow completed.")
            else:
                print("\nERROR: CACM workflow execution failed or had errors. Check logs.")

        except Exception as e:
            print(f"An error occurred during orchestrator execution: {e}")
            import traceback
            traceback.print_exc()

run_button.on_click(on_run_button_clicked)
display(run_button, output_area)


## 4. Results Display

In [ ]:
# Cell 10: Display Main Markdown Report
# This cell should be run after the "Run Analysis" button in Cell 8 has been clicked
# and has populated the `latest_analysis_results` global variable.

if 'latest_analysis_results' in globals() and latest_analysis_results and latest_analysis_results.get('final_generated_report'):
    print("Displaying Comprehensive Analysis Report (Markdown):")
    display(Markdown(latest_analysis_results['final_generated_report']))
else:
    print("No analysis results found. Please click 'Run Analysis' in Cell 8 first.")
    if 'latest_analysis_results' in globals() and latest_analysis_results:
        # If results exist but report is missing, print the raw output for debugging
        print("\nRaw outputs from orchestrator (in case of missing report):")
        print(json.dumps(latest_analysis_results, indent=2))

### 4.1. Detailed Agent Outputs (Raw JSON)
Below are the raw JSON outputs from the individual analytical agents.

In [ ]:
# Cell 12: Display Fundamental Analysis Output
if 'latest_analysis_results' in globals() and latest_analysis_results and latest_analysis_results.get('raw_fundamental_analysis'):
    print("Raw Output from Fundamental Analyst Agent:")
    display(JSON(latest_analysis_results['raw_fundamental_analysis']))
else:
    print("No Fundamental Analysis output found. Please run the analysis first.")

In [ ]:
# Cell 13: Display SNC Analysis Output
if 'latest_analysis_results' in globals() and latest_analysis_results and latest_analysis_results.get('raw_snc_analysis'):
    print("Raw Output from SNC Analyst Agent:")
    display(JSON(latest_analysis_results['raw_snc_analysis']))
else:
    print("No SNC Analysis output found. Please run the analysis first.")

In [ ]:
# Cell 14: Display Catalyst Insights Output
if 'latest_analysis_results' in globals() and latest_analysis_results and latest_analysis_results.get('raw_catalyst_output'):
    print("Raw Output from Catalyst Wrapper Agent:")
    display(JSON(latest_analysis_results['raw_catalyst_output']))
else:
    print("No Catalyst Insights output found. Please run the analysis first.")

## 5. Limitations and Future Work
- Currently, detailed financial data is only available for 'MSFT' and 'TESTCORP'. Analysis for other company IDs uses a generic placeholder dataset, providing illustrative rather than financially accurate results for those specific companies.
- The system does not perform full parsing of raw financial documents (e.g., 10-K PDFs). Financial data for `MSFT` and `TESTCORP` is pre-structured, and generic data is used otherwise.
- Textual analysis (e.g., for Business Overview, Risk Factors) currently uses static example text for `MSFT` or generic placeholders for other companies, loaded during the data ingestion phase of the demonstration workflow.
- Further enhancements could include:
    - Integration with live financial data providers.
    - Advanced NLP for extracting information from documents.
    - More sophisticated and configurable Semantic Kernel prompts for deeper, more nuanced AI-generated insights.
    - Interactive charts and visualizations of financial data and agent outputs.
    - A wider range of user-configurable parameters for each agent.